In [171]:
import pandas as pd
import elasticsearch
from elasticsearch_dsl import Search, A
from elasticsearch_dsl.query import MatchAll, Match

multiterm aggregation
#https://www.elastic.co/guide/en/elasticsearch/reference/current/search-aggregations-bucket-terms-aggregation.html

In [173]:
es = elasticsearch.Elasticsearch(hosts=[{'host': 'localhost', 'port': 9200}])

In [3]:
query = {
    "query": {
        "bool": {
            "must": [
                {
                    "multi_match": {
                        "query": "",
                        "fuzziness": "AUTO",
                        "fields": [
                            "phrase",#,#,
                            "phrase.ext"
                            #"phrase.bigrams"
                            #"phrase.ngram"
                        ]
                    }
                },
            ],
            "should": [
                {
                    "multi_match": {
                        "query": "",
                        "fuzziness": 1,
                        "fields": [
                            "norm_phrase"
                            #"norm_phrase.bigrams"
                            #"norm_phrase.ngram"
                        ]
                    }
                }
            ]
        }
    },
    "size": 25,
    "sort": {
        "_score": {
            "order": "desc"
        }
    },
}

query = {
    "query": {
        "match_all": {}
    }
}
res = es.search(index="rdrs110722", body = query)

/tmp/ipykernel_11825/3176541370.py:46: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  res = es.search(index="rdrs110722", body = query)


In [163]:
s = Search(using=es, index="rdrs110722").source(["Drugname_kw", "Diseasename_kw", "review_url_kw"]).query("match_all")

In [156]:
s = Search(using=es, index="rdrs110722").source(["Drugname_kw", "Diseasename_kw", "review_url_kw"]).aggs.bucket("UniqueDrugnames", agg_type="terms", field="Drugname_kw")

In [164]:
response = s.execute()

In [165]:
listed = []
#for hit in s[0:3800]:
for hit in s.scan():
    listed.append(hit.to_dict())

In [148]:
res = s.aggs.bucket("UniqueDrugnames", agg_type="terms", field="Drugname_kw").bucket("UniqueDrugnames", agg_type="sum", field="Diseasename_kw"


In [170]:
s = Search(using=es, index="rdrs110722").source(["Drugname_kw", "Diseasename_kw", "review_url_kw"]).query("match_all")
s.aggs.bucket("drugs", A("cardinality", field="Drugname_kw"))
res = s.execute()

RequestError: RequestError(400, 'search_phase_execution_exception', 'Text fields are not optimised for operations that require per-document field data like aggregations and sorting, so these operations are disabled by default. Please use a keyword field instead. Alternatively, set fielddata=true on [Drugname_kw] in order to load field data by uninverting the inverted index. Note that this can use significant memory.')

In [123]:
s["UniqueDrugnames"].bucket("tags_per_category")

TypeError: '<' not supported between instances of 'str' and 'int'

In [166]:
pd.DataFrame(listed)

,Drugname_kw,Diseasename_kw
0,[Амиксин],"[герпеса, Герпес]"
1,"[Иммунофлазид, Иммунофлазид]","[ГРВИ, грипа]"
2,"[Гавискон, Гавискон, Гавискон]",NaN
3,[Феназепам],NaN
4,[Виферон],[простуд]
...,...,...
3816,"[Кленбутерол, кленбутерол]",NaN
3817,"[Леденцы Вулкан Лакричные, леденцах лакричных,...",NaN
3818,"[Диафлекс РОМФАРМ, Диафлекс, омепрозол, Диафле...",NaN
3819,"[Целебрекс, цереблекс]",[обострении радикулита]


# простой индекс

In [221]:
#s = Search(using=es, index="rdrs110722_scheme2").query("match_all")
#s = Search(using=es, index="rdrs110722_scheme2").query("match", Indications="кашель, насморк")
s = Search(using=es, index="med800k_scheme2").query("match_all").count()
s = Search(using=es, index="med800k_scheme2").query("match", Indications="кашель, насморк").count()

In [222]:
s

23859

In [213]:
res = s.execute()
res

AttributeError: 'int' object has no attribute 'execute'

In [195]:
listed = []
#for hit in s[0:3800]:
for hit in s.scan():
    listed.append(hit.to_dict())

KeyboardInterrupt: 

In [188]:
pd.DataFrame(listed)

,Drugnames,Diseasenames,Indications,ADRs,ADR_reviews_count,Negated_ADE_reviews_count,Neg_reviews_count,Pos_reviews_countg,Review_count,Review_urls
0,"бактериофагом, виферон, кипферон, кипферона, с...",,"кашлем, насморк, понос, поносили, температура","дермит, на щёчке и мягкой части попы красные п...",1,1,1,0,1,[http://otzovik.com/review_395800.html]
1,"антигриппин, антигриппина, иммунал","аллергическим ринитом, гриппа, гриппе, простуд...","насморку, сильный насморк, слезотечение, слезы...",,0,0,0,1,1,[http://otzovik.com/review_396905.html]
2,ринофлуимуцил,"бронхит, простуда, простуде","кашель, насморки",,0,0,0,1,1,[http://otzovik.com/review_3169090.html]
3,"аквалор, аквалор baby, аквалор беби",,насморк,,0,0,0,1,1,[http://otzovik.com/review_407963.html]
4,ксимелин,,"заложенности, насморк","привыкание, слизистую сушит",1,0,1,1,1,[http://otzovik.com/review_413118.html]
...,...,...,...,...,...,...,...,...,...,...
298,"виферон, вифероном","герпесе, гриппа, орви","насморк, профилактики",,0,0,0,1,1,[http://otzovik.com/review_610650.html]
299,"ингаверина, ингавирин, капотеном","орви, простуды","горло, насморк",поднялось давление 145/90,1,1,1,1,1,[http://otzovik.com/review_6247440.html]
300,"тонзилгон, тонзилгон н, тонзилгоном",трахеит,"больного горла, кашель",,0,0,0,1,1,[http://otzovik.com/review_643275.html]
301,назоферон,,"кашель, температура, чихать",,0,1,1,0,1,[http://otzovik.com/review_644737.html]


In [193]:
import json
with open("/s/ls4/users/grartem/SAG_MED/med-corpus-analyser/data/raw/ner_800k_jsonl/ner_800k.jsonl", "r") as f:
    for line in f:
        data = json.loads(line)
        print(data)
        break

{'text_id': 0, 'text': 'Я люблю пользоваться всякими гелями для душа, особенно, которые приятно пахнут. Кремом-гелем пользовалась впервые. Его нам подарили на работе на 8 марта. Что могу сказать-отличный подарок. Раньше продукцией Dove я практически не пользовалась, разве только мылом, крем-гель мне очень понравился. Нежный, пенящийся, приятный легкий аромат. Не сушит кожу после использования, хорошо увлажняет и питает. Очень вкусно пахнет кокосом и жасмином, ненавязчиво. Пользоваться-сплошное наслаждение. Теперь я проверила его действие на себе и смело могу купить в подарок близким людям и друзьям. Могу порекомендовать крем-гель Dove всем любительницам понежеться в ванне, я думаю, что вам он тоже понравится.', 'entities': {'0': {'entity_id': 0, 'text': 'очень понравился кожу', 'tag': ['Disease:DisTypeBNE-Pos'], 'spans': [{'begin': 277, 'end': 293}]}, '1': {'entity_id': 1, 'text': 'хорошо увлажняет и питает', 'tag': ['Disease:DisTypeBNE-Pos'], 'spans': [{'begin': 373, 'end': 398}]}, '2

In [191]:
data

{'text_id': 0,
 'text': 'Я люблю пользоваться всякими гелями для душа, особенно, которые приятно пахнут. Кремом-гелем пользовалась впервые. Его нам подарили на работе на 8 марта. Что могу сказать-отличный подарок. Раньше продукцией Dove я практически не пользовалась, разве только мылом, крем-гель мне очень понравился. Нежный, пенящийся, приятный легкий аромат. Не сушит кожу после использования, хорошо увлажняет и питает. Очень вкусно пахнет кокосом и жасмином, ненавязчиво. Пользоваться-сплошное наслаждение. Теперь я проверила его действие на себе и смело могу купить в подарок близким людям и друзьям. Могу порекомендовать крем-гель Dove всем любительницам понежеться в ванне, я думаю, что вам он тоже понравится.',
 'entities': {'0': {'entity_id': 0,
   'text': 'очень понравился кожу',
   'tag': ['Disease:DisTypeBNE-Pos'],
   'spans': [{'begin': 277, 'end': 293}]},
  '1': {'entity_id': 1,
   'text': 'хорошо увлажняет и питает',
   'tag': ['Disease:DisTypeBNE-Pos'],
   'spans': [{'begin': 